# 20160221-predict-household-income-from-census

Related post:  
https://stharrold.github.io/20160221-predict-household-income-from-census.html

Data documentation:
https://www.census.gov/programs-surveys/acs/technical-documentation/pums/documentation.2013.html

## Initialization

### Imports

In [1]:
cd ~

/home/samuel_harrold


In [20]:
# Import standard packages.
import collections
import functools
import json
import os
import pdb # Debug with pdb.
import pprint
import sys
import time
# Import installed packages.
import astroML.density_estimation as astroML_dens # TODO: remove
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn as sk
import sklearn.ensemble as sk_ens
import seaborn as sns
# Import local packages.
# Insert current directory into module search path.
# Autoreload local packages after editing.
# `dsdemos` version: https://github.com/stharrold/dsdemos/releases/tag/v0.0.5
sys.path.insert(0, os.path.join(os.path.curdir, r'dsdemos'))
%reload_ext autoreload
%autoreload 2
import dsdemos as dsd
%matplotlib inline

In [19]:
print("Timestamp:")
print(time.strftime(r'%Y-%m-%dT%H:%M:%S%Z', time.gmtime()))
print()
print("Versions:")
print("Python:", sys.version_info)
print("matplotlib:", mpl.__version__)
print("numpy:", np.__version__)
print("pandas:", pd.__version__)
print("seaborn:", sns.__version__)
print("sklearn:", sk.__version__)
print("dsdemos:", dsd.__version__)

Timestamp:
2016-02-08T23:24:06GMT

Versions:
Python: sys.version_info(major=3, minor=5, micro=1, releaselevel='final', serial=0)
matplotlib: 1.5.1
numpy: 1.10.4
pandas: 0.17.1
seaborn: 0.7.0
sklearn: 0.17
dsdemos: 0.0.5


### Globals

File sources:
* 2013 5-year PUMS data dictionary: [PUMS_Data_Dictionary_2009-2013.txt](http://www2.census.gov/programs-surveys/acs/tech_docs/pums/data_dict/PUMS_Data_Dictionary_2009-2013.txt) (<1&nbsp;MB)
* 2013 5-year PUMS person and housing records for Washington DC:
    * Person records: [csv_pdc.zip](http://www2.census.gov/programs-surveys/acs/data/pums/2013/5-Year/csv_pdc.zip) (5&nbsp;MB compressed, 30&nbsp;MB decompressed)
    * Housing records: [csv_hdc.zip](http://www2.census.gov/programs-surveys/acs/data/pums/2013/5-Year/csv_hdc.zip) (2&nbsp;MB compressed, 13&nbsp;MB decompressed)
* 2013 5-year PUMS estimates for user verification: [pums_estimates_9_13.csv](http://www2.census.gov/programs-surveys/acs/tech_docs/pums/estimates/pums_estimates_9_13.csv) (<1&nbsp;MB)

In [6]:
# State settings.
sns.set() # Set matplot lib styles with seaborn
np.random.seed(seed=0) # Set random state for reproducibility.

# File paths
path_static = os.path.join(os.path.expanduser(r'~'), r'stharrold.github.io/content/static')
basename = r'20160221-predict-household-income-from-census'
filename = basename
path_ipynb = os.path.join(path_static, basename, filename+'.ipynb')
path_disk = os.path.abspath(r'/mnt/disk-20151227t211000z/')
path_acs = os.path.join(path_disk, r'www2-census-gov/programs-surveys/acs/')
path_dtxt = os.path.join(path_acs, r'tech_docs/pums/data_dict/PUMS_Data_Dictionary_2009-2013.txt')
path_pcsv = os.path.join(path_acs, r'data/pums/2013/5-Year/ss13pdc.csv') # 'pdc' = 'person DC'
path_hcsv = os.path.join(path_acs, r'data/pums/2013/5-Year/ss13hdc.csv') # 'hdc' = 'housing DC'

# Weights
pwt = 'PWGTP' # person weight
pwts = [pwt+str(inum) for inum in range(1, 81)]
hwt = 'WGTP' # housing weight
hwts = [hwt+str(inum) for inum in range(1, 81)]

# Statistics
# percentiles = [-1.6449 sigma, -1 sigma, mean, +1 sigma, +1.6449 sigma] for normal distribution
# 90.00% confidence interval = (percentiles[0], percentiles[-1])
# 68.27% confidence interval = (percentiles[1], percentiles[-2])
# median = percentiles[2]
percentiles = [0.0500, 0.1587, 0.5000, 0.8413, 0.9500]

## Extract, transform, and load

Related post:  
https://stharrold.github.io/20160110-etl-census-with-python.html

### Data dictionary

In [7]:
print("`ddict`: Load the data dictionary.")
ddict = dsd.census.parse_pumsdatadict(path=path_dtxt)

`ddict`: Load the data dictionary.


### PUMS data

In [92]:
print("`dfp`, `dfh`: Load person and housing records.")
# For ss13pdc.csv, low_memory=False since otherwise pandas raises DtypeWarning.
dfp = pd.read_csv(path_pcsv, low_memory=False)
dfh = pd.read_csv(path_hcsv, low_memory=True)

`dfp`, `dfh`: Load person and housing records.


In [61]:
dfp_cols = set(ddict['record_types']['PERSON RECORD'].keys())
dfp_cols.difference_update(pwts)
dfp_cols.intersection_update(dfp.columns.values)
dfp_cols = list(dfp_cols)
dfp[dfp_cols].head()

,FMARP,POBP05,ANC2P05,MLPI,MLPB,FANCP,FDREMP,FGCMP,CITWP05,RAC1P,...,AGEP,MIGSP05,SEX,JWRIP,HINS3,ANC1P12,DECADE,SFR,DRATX,JWAP
0,0,34,50,NaN,NaN,0,0,0,NaN,1,...,38,NaN,2,NaN,2,-9,NaN,NaN,NaN,86
1,0,37,999,NaN,NaN,0,0,0,NaN,2,...,78,NaN,2,NaN,1,-9,NaN,NaN,NaN,NaN
2,0,11,999,NaN,NaN,0,0,0,NaN,2,...,39,NaN,2,1,2,-9,NaN,3,NaN,259
3,0,11,999,NaN,NaN,0,0,0,NaN,2,...,8,NaN,1,NaN,2,-9,NaN,5,NaN,NaN
4,0,12,750,NaN,NaN,0,0,0,NaN,2,...,53,NaN,1,1,2,-9,NaN,NaN,NaN,103


In [63]:
dfh_cols = set(ddict['record_types']['HOUSING RECORD'].keys())
dfh_cols.difference_update(hwts)
dfh_cols.intersection_update(dfh.columns.values)
dfh_cols = list(dfh_cols)
assert len(dfh) == len(dfh['SERIALNO'].unique())
dfh[dfh_cols].head()

,FTELP,FVALP,HUPAOC,ELEP,NPP,FTOILP,SRNT,FYBLP,BLD,HHT,...,FWATP,ADJHSG,FSINKP,SERIALNO,CONP,R18,PSF,FMRGP,HUGCL,DIVISION
0,0,0,4,40,0,0,0,0,8,6,...,0,1086032,0,2009000000403,240,0,0,0,0,5
1,0,0,4,250,0,0,1,0,3,3,...,0,1086032,0,2009000001113,0,1,1,0,1,5
2,0,1,4,80,0,0,0,0,2,2,...,1,1086032,0,2009000001978,0,0,0,1,0,5
3,0,0,4,60,0,0,1,0,9,5,...,0,1086032,0,2009000002250,0,0,0,0,0,5
4,0,0,4,140,0,0,0,0,2,1,...,0,1086032,0,2009000002985,0,0,0,0,0,5


In [218]:
# Only keep households for which there are person records.
df = pd.merge(left=dfp[dfp_cols], right=dfh[dfh_cols], how='left', on='SERIALNO', suffixes=('_p', '_h'))
assert len(df) == len(dfp)
# Cast all types to float and replace string values.
# TODO: Useful link: For learning and testing regular expressions: http://regexr.com/
# TODO: Useful link: For comparing SQL and pandas 'JOIN' operations: http://pandas.pydata.org/pandas-docs/stable/merging.html
obj_cols = df.dtypes[df.dtypes == 'object'].index
for col in obj_cols:
    # 'RT' record types.
    if 'RT' in col:
        df[col] = df[col].apply(ord)
    # 'SOC' occupational codes.
    if 'SOCP' in col:
        df[col] = df[col].str.replace('X', '0')
        df[col] = df[col].str.replace('Y', '0')
        df[col] = df[col].replace(to_replace=r'N\.A\.(\/\/)?', value=0, inplace=False, regex=True)
    # 'OCC' occupational codes.
    if 'OCCP' in col:
        df[col] = df[col].replace(to_replace=r'N\.A\.(\/\/)?', value=0, inplace=False, regex=True)
    # 'NAICSP' industry codes.
    if 'NAICSP' in col:
        df[col] = df[col].str.replace(r'\D', '0')
        ndigits = df[col].apply(lambda obj: len(str(obj))).max()
        df[col] = df[col].str.ljust(ndigits, '0')
    df[col] = df[col].astype(float)
df = df.astype(float)

In [215]:
col

'OCCP10'

In [208]:
df.loc[:10, df.columns.values[:20]]

,FMARP,POBP05,ANC2P05,MLPI,MLPB,FANCP,FDREMP,FGCMP,CITWP05,RAC1P,GCM,ESR,MIGPUMA10,FRETP,FHINS4P,FPOWSP,ANC,FCITWP,DEYE,FDRATP
0,0,34,50,NaN,NaN,0,0,0,NaN,1,NaN,1,NaN,1,0,0,2,0,2,0
1,0,37,999,NaN,NaN,0,0,0,NaN,2,NaN,6,NaN,1,0,0,1,0,2,0
2,0,11,999,NaN,NaN,0,0,0,NaN,2,NaN,1,NaN,0,0,0,1,0,2,0
3,0,11,999,NaN,NaN,0,0,0,NaN,2,NaN,NaN,NaN,0,0,0,1,0,2,0
4,0,12,750,NaN,NaN,0,0,0,NaN,2,NaN,1,NaN,0,0,0,2,0,2,0
5,0,12,999,NaN,NaN,0,0,0,NaN,2,NaN,1,NaN,0,0,0,1,0,2,0
6,0,29,999,NaN,NaN,0,0,0,NaN,1,NaN,1,-9,0,0,0,1,0,2,0
7,0,36,51,NaN,NaN,0,0,0,NaN,1,NaN,1,NaN,0,0,0,2,0,2,0
8,0,36,51,NaN,NaN,0,0,0,NaN,1,NaN,1,-9,0,0,0,2,0,2,0
9,0,36,999,NaN,NaN,0,0,0,NaN,1,NaN,1,NaN,0,0,0,1,0,2,0


In [209]:
df.loc[:10, df.columns.values[20:40]]

,SOCP00,ST_p,PERNP,FRELP,NWLK,FOCCP,MIL,MARHM,PUBCOV,FRACP,SSP,RACBLK,FINTP,FSSP,SCIENGP,FDDRSP,MLPCD,SOCP10,FHINS2P,FCITP
0,119100,11,42000,0,2,1,4,NaN,2,0,0,0,1,1,1,0,NaN,NaN,0,0
1,NaN,11,0,0,2,0,4,2,1,0,0,1,0,0,NaN,0,NaN,NaN,0,0
2,292030,11,80000,0,3,0,4,NaN,2,0,0,1,0,0,NaN,0,NaN,NaN,0,0
3,NaN,11,NaN,0,NaN,0,NaN,NaN,2,0,NaN,1,0,0,NaN,0,NaN,NaN,0,0
4,231000,11,130000,0,3,0,4,2,2,0,0,1,1,0,2,0,NaN,NaN,0,0
5,232090,11,36900,0,3,0,4,NaN,2,0,0,1,0,0,NaN,0,NaN,NaN,0,0
6,119100,11,30000,0,3,0,4,NaN,2,0,0,0,0,0,2,0,NaN,NaN,0,0
7,253000,11,75000,0,3,0,4,NaN,2,0,0,0,0,0,2,0,NaN,NaN,0,0
8,273041,11,52000,0,3,0,4,NaN,2,0,0,0,0,0,2,0,NaN,NaN,0,0
9,291020,11,150000,0,3,0,4,2,2,0,0,0,0,0,1,0,NaN,NaN,0,0


In [224]:
ddict['record_types']['PERSON RECORD']['NAICSP']

OrderedDict([('length', '8'),
             ('description', 'NAICS Industry code based on 2012 NAICS codes'),
             ('var_codes',
              OrderedDict([('bbbbbbbb',
                            'Not in universe (less than 16 years old/NILF who last worked more than 5 years ago or never worked)'),
                           ('111     ', 'AGR-CROP PRODUCTION'),
                           ('112     ',
                            'AGR-ANIMAL PRODUCTION AND AQUACULTURE'),
                           ('1133    ', 'AGR-LOGGING'),
                           ('113M    ', 'AGR-FORESTRY EXCEPT LOGGING'),
                           ('114     ', 'AGR-FISHING, HUNTING, AND TRAPPING'),
                           ('115     ',
                            'AGR-SUPPORT ACTIVITIES FOR AGRICULTURE AND FORESTRY'),
                           ('211     ', 'EXT-OIL AND GAS EXTRACTION'),
                           ('2121    ', 'EXT-COAL MINING'),
                           ('2122    ', 'EXT-METAL ORE M

## Select features

In [ ]:
model = sk.feature_selection.SelectFromModel(
    estimator=sk_ens.ExtraTreesRegressor(), threshold='mean', prefit=False)
model.fit()

# TODO: redo below

## Select features

**Notes:**
* Example consumer databases: http://www.consumerreports.org/cro/money/consumer-protection/big-brother-is-watching/overview/index.htm?rurl=http%3A%2F%2Fwww.consumerreports.org%2Fcro%2Fmoney%2Fconsumer-protection%2Fbig-brother-is-watching%2Foverview%2Findex.htm
* Random forests are scale invariant, so they can accommodate non-linear transformation.
* Cast all values to floats so that compatable with most algorithms and can use </> logic. Otherwise less informationally dense and may require deeper tree structure to find features.
* To "map to float" ('b' is N/A, mapped to 0; 1 is Yes; 2 is No; other values are special):  
```python
test = pd.DataFrame(data=[['  b', 1.0], ['1', 1.0], ['2', 1.0], ['3', 1.1], ['4', 1.1]], columns=['COL', 'ADJ'])
tfmask = test['COL'].str.contains('b')
test.loc[tfmask, 'COL'] = 0.0
test['COL'] = test['COL'].astype(float)
print(test.dtypes)
test
```
* To "adjust for inflation":  
```python
test['ADJ'] *= 1e-6
tfmask = test['COL'] >= 3.0
test.loc[tfmask, 'COL'] *= test.loc[tfmask, 'ADJ']
test
```
* TODO: Remove vacant units ('NP') from data frame.
* TODO: Filter categorical variables from metadata (those without '..').

In [66]:
record_type = 'PERSON RECORD'
print_detail = False
for key in ddict['record_types'][record_type]:
    desc = ddict['record_types'][record_type][key]['description']
    if not (
        (key.startswith('F') and (desc.endswith(' flag') or desc.endswith(' edit')))
        or ('WGTP' in key and "Weight replicate" in desc)):
        if print_detail:
            print(key)
            pprint.pprint(ddict['record_types'][record_type][key])
        else:
            print("{key}: {desc}".format(key=key, desc=desc))

RT: Record Type
SERIALNO: Housing unit/GQ person serial number
SPORDER: Person number
PUMA00: Public use microdata area code (PUMA) based on Census 2000 definition for data collected prior to 2012. Use in combination with PUMA10.
PUMA10: Public use microdata area code (PUMA) based on 2010 Census definition for data Collected in 2012 or later. Use in combination with PUMA00.
ST: State Code
ADJINC: Adjustment factor for income and earnings dollar amounts (6 implied decimal places)
PWGTP: Person's weight
AGEP: Age
CIT: Citizenship status
CITWP05: Year of naturalization write-in for data collected prior to 2012
CITWP12: Year of naturalization write-in for data collected in 2012 or later
COW: Class of worker
DDRS: Self-care difficulty
DEAR: Hearing difficulty
DEYE: Vision difficulty
DOUT: Independent living difficulty
DPHY: Ambulatory difficulty
DRAT: Veteran service connected disability rating (percentage)
DRATX: Veteran service connected disability rating (checkbox)
DREM: Cognitive diffic

In [85]:
# Include columns that I think companies can easily get.
# for column details: https://www.census.gov/programs-surveys/acs/technical-documentation/pums/documentation.2013.html
# target: HINCP

cols_include = {
    'HOUSING RECORD': [
        'SERIALNO', 'PUMA00', 'PUMA10', 'ST', 'ADJHSG', 'ADJINC', 'WGTP', 'NP', 'BDSP', 'BLD', 'HINCP', 'R18', 'R65'],
    'PERSON RECORD': [
        'SERIALNO', 'SPORDER', 'PWGTP', 'AGEP', 'MAR', 'SCHL', 'INDP']
    }

In [86]:
for record_type in cols_include:
    print(record_type)
    for var_name in cols_include[record_type]:
        desc = ddict['record_types'][record_type][var_name]['description']
        print("{var}: {desc}".format(var=var_name, desc=desc))
    print()

HOUSING RECORD
SERIALNO: Housing unit/GQ person serial number
PUMA00: Public use microdata area code (PUMA) based on Census 2000 definition for data collected prior to 2012. Use in combination with PUMA10.
PUMA10: Public use microdata area code (PUMA) based on 2010 Census definition for data collected in 2012 or later. Use in combination with PUMA00.
ST: State Code
ADJHSG: Adjustment factor for housing dollar amounts (6 implied decimal places)
ADJINC: Adjustment factor for income and earnings dollar amounts (6 implied decimal places)
WGTP: Housing Weight
NP: Number of person records following this housing record
BDSP: Number of bedrooms
BLD: Units in structure
HINCP: Household income (past 12 months)
R18: Presence of persons under 18 years in household (unweighted)
R65: Presence of persons 65 years and over in household (unweighted)

PERSON RECORD
SERIALNO: Housing unit/GQ person serial number
SPORDER: Person number
PWGTP: Person's weight
AGEP: Age
MAR: Marital status
SCHL: Educational

In [80]:
record_type = 'PERSON RECORD'
var_codes = ddict['record_types'][record_type]['INDP']['var_codes']
indp_abbr = dict()
for var_code in var_codes.keys():
    indp_abbr[var_code] = var_codes[var_code].split(sep='-', maxsplit=1)[0][:3]
print(sorted(set(indp_abbr.values())))

['ADM', 'AGR', 'CON', 'EDU', 'ENT', 'EXT', 'FIN', 'INF', 'MED', 'MFG', 'MIL', 'Not', 'PRF', 'RET', 'SCA', 'SRV', 'TRN', 'UNE', 'UTL', 'WHL']


Actions for included columns:  
* HOUSING RECORD
    * SERIALNO: Use to join to PERSON RECORD.
    * PUMA00, PUMA10, ST: Combine and lookup lat-lon coordinates from census.gov.
    * ADJHSG, ADJINC: Multiply against other columns to adjust for inflation. See https://www.census.gov/library/publications/2009/acs/pums.html App 5.
    ADJHSG: CONP, ELEP, FULP, GASP, GRNTP, INSP, MHP, MRGP, SMOCP, RNTP, SMP, WATP
    ADJINC: INTP, OIP, PAP, PERNP, PINCP, RETP, SEMP, SSIP, SSP, WAGP
    * WGTP: Confirm with user verification file.
    * NP: Numerical. Map to float.
    * BDSP: Numerical. Map to float.
    * BLD: Map to median income.
    * HINCP: Include. Map household income to float. Adjust all with ADJINC.
    * R18: Include. Presence of persons under 18 years in household. Map to float.
    * R65: Include. Presence of persons 60+ years in household.
* PERSON RECORD
    * SERIALNO, SPORDER: Use as index.
    * PWGTP: Confirm user verification file.
    * AGEP: Map to float.
    * MAR: Map to float.
    * SCHL: Map to float.
    * INDP: (1) Map to float. (2) Map categories to median income.

## Export ipynb to html

In [158]:
!date --rfc-3339='seconds'

2016-01-01 05:40:17+00:00


In [ ]:
path_ipynb = os.path.join(path_static, basename, basename+'.ipynb')
for template in ['basic', 'full']:
    path_html = os.path.splitext(path_ipynb)[0]+'-'+template+'.html'
    cmd = ['jupyter', 'nbconvert', '--to', 'html', '--template', template, path_ipynb, '--output', path_html]
    print(' '.join(cmd))
    subprocess.run(args=cmd, check=True)